In [61]:
import pandas as pd
import numpy as np
import lightgbm
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, accuracy_score
from sklearn.model_selection import GridSearchCV

In [31]:
base_path = "../data/dataframes/"
data = pd.read_pickle(base_path + 'data.pkl')
nomination_onehot = pd.read_pickle(base_path + 'nomination_onehot.pkl')
selected_performers_onehot = pd.read_pickle(base_path + 'selected_performers_onehot.pkl')
selected_directors_onehot = pd.read_pickle(base_path + 'selected_directors_onehot.pkl')
selected_studio_onehot = pd.read_pickle(base_path + 'selected_studio_onehot.pkl')
selected_scriptwriter_onehot = pd.read_pickle(base_path + 'selected_scriptwriter_onehot.pkl')

data.tail()

,director,other_nominates,performers,prize,production_studio,screen_time,scriptwriter,title,year,selected_performers,selected_directors,selected_studio,selected_scriptwriter
id,,,,,,,,,,,,,
207,"[是枝裕和, 森本晶一]","[nikkan_sports, mainichi_film_award, kinejun_b...","[リリー・フランキー, 安藤サクラ, 松岡茉優, 城桧吏, 佐々木みゆ, 池松壮亮, 山田裕...",1,[ギャガ],120,[是枝裕和],万引き家族,2019,"[柄本明, 樹木希林]",[是枝裕和],[],[是枝裕和]
208,"[上田慎一郎, 中泉裕矢, 吉田幸之助]",[blue_ribbon_award],"[濱津隆之, 真魚, しゅはまはるみ, 長屋和彰, 細井学, 市原洋, 山崎俊太郎, 大澤真...",0,"[アスミック・エース, ENBUゼミナール]",96,[上田慎一郎],カメラを止めるな！,2019,[],[],[],[]
209,"[滝田洋二郎, 浜田毅, 足立公良]",[],"[吉永小百合, 堺雅人, 篠原涼子, 岸部一徳, 高島礼子, 永島敏行, 安田顕, 野間口徹...",0,[東映],126,[那須真知子],北の桜守,2019,"[岸部一徳, 佐藤浩市, 吉永小百合, 永島敏行]","[滝田洋二郎, 足立公良]",[東映],[那須真知子]
210,"[白石和彌, 山本亮]",[hochi_eigashou],"[役所広司, 松坂桃李, 真木よう子, 滝藤賢一, 音尾琢真, 駿河太郎, 中村倫也, 中村...",0,[東映],126,[池上純哉],孤狼の血,2019,"[石橋蓮司, 役所広司]",[],[東映],[]
211,"[本木克英, 向井澄]",[],"[長瀬智也, ディーン・フジオカ, 高橋一生, 深田恭子, 岸部一徳, 笹野高史, 寺脇康文...",0,[松竹],120,[林民夫],空飛ぶタイヤ,2019,"[柄本明, 笹野高史, 岸部一徳]",[],[松竹],[]


In [27]:
df = pd.concat([nomination_onehot, selected_performers_onehot, selected_directors_onehot, selected_studio_onehot, selected_scriptwriter_onehot, data["screen_time"]],
               axis=1)

In [45]:
y = data["prize"].values
x = df.values

In [46]:
#
# Create training and validation sets
#
x, x_test, y, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

In [50]:
#
# Create the LightGBM data containers
#
train_data = lightgbm.Dataset(x, label=y) # TODO: set categorical features
test_data = lightgbm.Dataset(x_test, label=y_test)

In [51]:
#
# Train the model
#

parameters = {
    'application': 'binary',
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'true',
    'boosting': 'gbdt',
    'num_leaves': 31,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'learning_rate': 0.05,
    'verbose': 0
}

In [60]:
model = lightgbm.train(parameters,
                       train_data,
                       valid_sets=test_data,
                       num_boost_round=5000,
                       early_stopping_rounds=5000
)

[1]	valid_0's auc: 0.562092
Training until validation scores don't improve for 5000 rounds.
[2]	valid_0's auc: 0.562092
[3]	valid_0's auc: 0.565359
[4]	valid_0's auc: 0.565359
[5]	valid_0's auc: 0.565359
[6]	valid_0's auc: 0.565359
[7]	valid_0's auc: 0.565359
[8]	valid_0's auc: 0.565359
[9]	valid_0's auc: 0.565359
[10]	valid_0's auc: 0.565359
[11]	valid_0's auc: 0.565359
[12]	valid_0's auc: 0.565359
[13]	valid_0's auc: 0.565359
[14]	valid_0's auc: 0.565359
[15]	valid_0's auc: 0.565359
[16]	valid_0's auc: 0.565359
[17]	valid_0's auc: 0.565359
[18]	valid_0's auc: 0.565359
[19]	valid_0's auc: 0.565359
[20]	valid_0's auc: 0.565359
[21]	valid_0's auc: 0.565359
[22]	valid_0's auc: 0.565359
[23]	valid_0's auc: 0.513072
[24]	valid_0's auc: 0.513072
[25]	valid_0's auc: 0.513072
[26]	valid_0's auc: 0.513072
[27]	valid_0's auc: 0.513072
[28]	valid_0's auc: 0.513072
[29]	valid_0's auc: 0.513072
[30]	valid_0's auc: 0.513072
[31]	valid_0's auc: 0.513072
[32]	valid_0's auc: 0.513072
[33]	valid_0's au

In [62]:
y = model.predict(x)
y

array([0.62854131, 0.34078149, 0.35289169, 0.72875955, 0.62854131,
       0.13549372, 0.62854131, 0.48660765, 0.61537948, 0.67943313,
       0.54139922, 0.72875955, 0.37467869, 0.4424978 , 0.48660765,
       0.49280654, 0.20109622, 0.37467869, 0.48660765, 0.33924144,
       0.37467869, 0.71392827, 0.3426839 , 0.48660765, 0.35554288,
       0.55794301, 0.62854131, 0.54139922, 0.54139922, 0.16567055,
       0.47189355, 0.3445277 , 0.47189355, 0.38780583, 0.35554288,
       0.41797093, 0.57395809, 0.34078149, 0.47189355, 0.27942854,
       0.44993068, 0.47189355, 0.21874416, 0.66154371, 0.86165167,
       0.41797093, 0.41797093, 0.53374446, 0.34078149, 0.29196731,
       0.208835  , 0.54411099, 0.34292438, 0.79939692, 0.68451794,
       0.54486356, 0.09161151, 0.34078149, 0.34078149, 0.47189355,
       0.61064506, 0.62854131, 0.41797093, 0.37467869, 0.41797093,
       0.09161151, 0.3445277 , 0.37467869, 0.61064506, 0.62854131,
       0.54411099, 0.42278188, 0.41797093, 0.48660765, 0.36942